#### DNN기반 다중분류 모델 구현 + 학습 진행 모니터링 + 저장
 - 데이터셋 : iris.csv
 - feature : 4개 Sepal_Length, Sepal_Width, Petal_Length, Petal_Width
 - target/label : 1개 variety
 - 학습방법 : 지도학습 > 분류 > 다중분류 (클래스 3개)
 - 알고리즘 : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
 - 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비
***

In [4]:
# - 모델 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.classification import MulticlassF1Score

# - 데이터 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [5]:
# 활용 패키지 버전 체크  ==> 사용자 정의 함수로 구현해놓기
def checkversion() :
    print(f'Pytorch v {torch.__version__}')
    print(f'pandas v {pd.__version__}')

In [7]:
#### 데이터 로딩
irisDF = pd.read_csv(r'C:\Users\hoon\Desktop\KDT 6\EX_PANDAS6-main\TORCH_DL\data\iris.csv')
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [8]:
### 타겟 변경 ==> 정수화, 클래스3개 => 2개
labels = dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f'labels => {labels}')

irisDF['variety'] = irisDF['variety'].replace(labels)

labels => {'Setosa': 0, 'Versicolor': 1, 'Virginica': 2}


[2] 모델 클래스 설계 및 정의<hr>
 - 클래스 목적 : iris 데이터 학습 및 추론
 - 클래스 이름 : IrisMCFModel
 - 부모 클래스 : nn.Module
 - 매개변수    : 층별 입/출력 개수 고정 => 매개변수 필요 없음 
 - 속성/필드   :  
 - 클래스 기능 : _ _ init _ _() : 모델 구조 설정, forward() : 순방향 학습  <= 오버라이딩(overriding)
 - 클래스 구조 
   * 입력층 : 입력 4개     출력 10개 (퍼셉트론 10개 존재)
   * 은닉층 : 입력 10개    출력 5개  (퍼셉트론 5개 존재)
   * 출력층 : 입력 5개     출력 1개 (다중분류)

 - 활성화함수
   * 클래스 형태 ==> nn.MELoss, nn.ReLU  -> _ _ init _ _() 메서드에서 사용
   * 함수 형태  ==> torch.nn.functional 아래에 -> forward() 메서드에서 사용

In [9]:
class irisMCFModel(nn.Module) :

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self) :
        super().__init__()
        self.in_layer = nn.Linear(4,10)
        self.h_layer = nn.Linear(10,5)
        self.out_layer = nn.Linear(5,3)     # 다중분류 'Setosa', 'Ver', 'Vir'

    # 순방향 학습 진행 메서드
    def forward(self, data) :
        # 입력층
        y = self.in_layer(data)             # f1w1+f2w2+f3w3+...+f10w10+b
        F.relu(y)                           # relu => y 값의 범위 : 0 < y
        
        # 은닉층 : 10개의 숫자값이 들어옴(>=0)
        y = self.h_layer(y)
        F.relu(y)

        # 출력층 : 5개의 숫자 값 => 다중분류 : 손실함수 crossEntropyLoss 내부에서 Softmax 처리 해서 pytorch에서는 안넣어도 됨
        return self.out_layer(y)

In [10]:
### 모델 인스턴스 생성
model = irisMCFModel()
print(model)

irisMCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (h_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=3, bias=True)
)


In [12]:
### [테스트] 모델 사용 메모리 정보 확인
# summary(model, input_size=(10,4))

[3] 데이터셋 클래스 설계 및 정의 <hr>

In [13]:
class IrisDataset(Dataset) :

    def __init__(self,featureDF, targetDF) :
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self) :
        return self.n_rows

    def __getitem__(self, index) :
        
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)

        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [24]:
## [테스트] 데이터셋 인스턴스 생성

# - DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1:]]


# - 커스텀 데이터셋 인스턴스 생성
irisDS = IrisDataset(featureDF,targetDF)

# - 데이터로더 인스턴스 생성        <= DataLoader로 테스트 해야함 for문으로 feature랑 label 하나씩 뽑아서 확인하믄 됨
irisDL = DataLoader(irisDS)
for feature, label in irisDL :
    print(feature.shape, label.shape)
    break

targetDF

torch.Size([1, 4]) torch.Size([1, 1])


,variety
0,0
1,0
2,0
3,0
4,0
...,...
145,2
146,2
147,2
148,2


[4] 학습 준비 <hr>
 - 학습횟수 : EPOCH          <- 처음 ~ 끝까지 공부하는 단위  
 - 배치크기 : BATCH_SIZE     <- 한번에 학습할 데이터 양
 - 위치지정 : DEVICE         <- 텐서 저장 및 실행 위치 (GPU/CPU)
 - 학습률(lr) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 (0.001 ~ 0.1) <- 하이퍼파라미터

In [15]:
### 학습 진행 관련 설정값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

In [16]:
print(DEVICE)

cpu


 - 학습 준비에 필요한 것
    * 인스턴스/객체 : 모델, 데이터셋, 최적화, (, 손실함수, 성능지표)

In [17]:
# 찐 모델 인스턴스 생성
model = irisMCFModel()

# - 학습용, 검증용, 테스트용 데이터셋 분리
X_train, X_test, y_train, y_test = train_test_split(featureDF,targetDF,random_state = 1)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,random_state = 1)

# - 학습용, 검증용, 테스트용 인스턴스 생성
trainDS = IrisDataset(X_train,y_train)
valDS = IrisDataset(X_val,y_val)
testDS = IrisDataset(X_test, y_test)

# - 학습용 데이터로더 인스턴스 생성
trainDL = DataLoader(trainDS,batch_size=BATCH_SIZE)

# 최적화 인스턴스 - 최소의 손실을 만드는 w,b 찾는 작업 해주는 인스턴스
# 최적화 인스턴스에 model.parameters() 전달 필요
optimizer = optim.Adam(model.parameters(),lr=LR)


# 손실함수 인스턴스 > 분류 > 다중분류 CrossEntropyLoss  
#                                  예측값은 선형식 결과값으로 전달 
crossLoss = nn.CrossEntropyLoss()

[5] 학습 진행<hr>

In [18]:
### models 폴더 아래 프로젝트 폴더 아래 모델 파일 저장
import os

# 저장 경로
SAVE_PATH = '../models/iris/'
# 저장 파일명
SAVE_FILE = 'model_train_wbs.pth'
# 모델 구조 및 파라미터 모두 저장 파일명
SAVE_MODEL='model_all.pth'

In [19]:
# 경로상 폴더 존재 여부 체크
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH) # 폴더/폴더/... 하위 폴더 까지 생성

In [20]:
## 학습의 효과 확인을 위해 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]],[[],[]]
CNT = len(trainDL)

## 학습 모니터링/스케줄링 설정 
# => LOSS_HISTORY, SCORE_HISTROY 활용
# => 임계기준 : 10번
BREAK_CNT = 0
LIMIT_VALUE = 10


for epoch in range(EPOCH) :
    # 학습 모드로 모델 설정
    model.train()


    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total , score_total = 0, 0
    for featureTS, targetTS in trainDL :
        
        # 학습 진행
        pre_y = model(featureTS)

        # 손실 계산 : nn.CrossEntropyLoss 요구사항 : 정답/타겟은 0D 또는 1D, 타입은 long
        loss = crossLoss(pre_y,targetTS.reshape(-1).long())
        loss_total += loss.item()

        # 성능 평가 계산
        score = MulticlassF1Score(num_classes=3)(pre_y,targetTS.reshape(-1))
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)
        # 추론/평가
        pre_val = model(val_featureTS)
        # 손실계산
        loss_val = crossLoss(pre_val, val_targetTS.reshape(-1).long())
        # 성능평가
        score_val = MulticlassF1Score(num_classes=3)(pre_val, val_targetTS.reshape(-1))



    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/CNT) 
    SCORE_HISTORY[0].append(score_total/CNT)
    LOSS_HISTORY[1].append(loss_val) 
    SCORE_HISTORY[1].append(score_val)  

      # 학습 진행 모니터링 / 스케줄링 - 검증 DS 기준 => Loss와 Score 중 둘중 한 개만 보면 됨.
    # Loss 기준
    if len(LOSS_HISTORY[0]) >= 2:
        if LOSS_HISTORY[1][-1] >= LOSS_HISTORY[1][-2]: BREAK_CNT += 1

     # 성능이 좋은 학습 가중치 저장
    SAVE_FILE = f'model_train_wbs_{epoch}_{score_val}.pth'
    if len(SCORE_HISTORY[1]) == 1:
        # 첫번째라서 무조건 모델 파라미터 저장
        torch.save(model.state_dict(),SAVE_FILE)
        # 모델 전체 저장
        torch.save(model,SAVE_MODEL)
    
    else:
        if SCORE_HISTORY[1][-1] > max(SCORE_HISTORY[1][:-1]):
            # 두번째 ==> 첫번째보다 성능 좋으면 저장
            torch.save(model.state_dict(),SAVE_FILE)
            torch.save(model,SAVE_MODEL)
    '''
    스코어의 경우 학습할수록 올라가야 좋은거지
    그래서 SCORE_HISTORY[1][-1]이 이전값들의 최고값보다 높아져야 성능이 올라갔다고 볼 수 있어
    
    '''

    # 학습 중단 여부 설정
    if BREAK_CNT > LIMIT_VALUE:
        print('성능 및 손실 개선이 없어서 학습 중단')
        break

성능 및 손실 개선이 없어서 학습 중단


In [21]:
print(LOSS_HISTORY[0])

[1.1226163572735257, 1.0779137346479628, 1.044663177596198, 1.0170706974135504, 0.9919198354085287, 0.9680083592732748, 0.9448988636334738, 0.9221756392055087, 0.8994007971551683, 0.8762336174647013, 0.8524585697386, 0.8279675642649332, 0.8027535941865709, 0.7769085102611117, 0.750607066684299, 0.7240804102685716, 0.697588152355618, 0.6713927057054307, 0.645738939444224, 0.620838032828437, 0.5968571636411879, 0.573913229836358, 0.5520730217297872, 0.5313577916887071, 0.511751393477122, 0.4932100905312432, 0.47567156619495815, 0.45906293723318314, 0.4433068533738454, 0.42832640475696987, 0.4140487379497952, 0.4004075626532237, 0.3873444332016839, 0.37480949362119037, 0.36276132199499345, 0.35116664899720085, 0.3399994373321533, 0.32923981381787193, 0.3188731024662654, 0.3088887698120541, 0.2992791583140691, 0.2900386568572786, 0.28116268581814235, 0.27264727817641365, 0.2644882880979114, 0.25668099688159096, 0.24922000037299263, 0.24209892418649462, 0.23531024323569405, 0.22884572048981